# 🚀 PhoBERT Fine-tuning on Kaggle (GPU)

## ⚡ Kaggle vs Colab:
| Feature | Kaggle | Google Colab |
|---------|--------|-------------|
| **GPU** | P100 (16GB) hoặc T4 (16GB) | T4 (15GB) |
| **RAM** | 30GB | 12GB |
| **Disk** | 73GB | 108GB |
| **Time limit** | 12 hours/week | Variable |
| **Tốc độ** | ⚡⚡⚡ Nhanh hơn | ⚡⚡ Trung bình |

## 📦 Thời gian ước tính:
- **P100 GPU**: ~20-30 phút (3 epochs)
- **T4 GPU**: ~30-40 phút (3 epochs)
- **Tăng tốc vs CPU**: ~60x nhanh hơn!

## 🎯 Bật GPU:
1. Click **Settings** (⚙️) bên phải
2. **Accelerator** → chọn **GPU P100** hoặc **GPU T4**
3. Click **Save**

## 📥 Bước 1: Clone Repository

In [ ]:
# Clone repo từ GitHub
!git clone https://github.com/d0ngle8k/NLP-Processing.git
%cd NLP-Processing

# Xem cấu trúc
!ls -la

## 📦 Bước 2: Cài Dependencies

In [ ]:
# Cài đặt packages cần thiết
!pip install -q torch transformers underthesea tqdm

print("✅ Dependencies installed!")

## 🎮 Bước 3: Kiểm Tra GPU

In [ ]:
import torch

print("="*60)
print("🔍 GPU Information")
print("="*60)
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"✅ CUDA Version: {torch.version.cuda}")
    print("\n🚀 Ready to train!")
else:
    print("\n⚠️  No GPU detected!")
    print("👉 Nhớ bật GPU: Settings → Accelerator → GPU P100")

print("="*60)

## 📊 Bước 4: Kiểm Tra Training Data

In [ ]:
import json
import os

# Kiểm tra file tồn tại
train_file = "training_data/phobert_training_augmented.json"

if os.path.exists(train_file):
    # Load và đếm samples
    with open(train_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    total = len(data)
    train_split = int(0.8 * total)
    
    print(f"✅ Training data found!")
    print(f"📊 Total samples: {total:,}")
    print(f"📊 Training: {train_split:,} (80%)")
    print(f"📊 Validation: {total - train_split:,} (20%)")
    print(f"\n📝 Sample data:")
    print(json.dumps(data[0], indent=2, ensure_ascii=False))
else:
    print(f"❌ Training data not found: {train_file}")
    print("👉 Check if file exists in repo")

## 🚀 Bước 5: Bắt Đầu Training

### ⚙️ Configuration:
- **Epochs**: 3 (có thể tăng lên 5-10 nếu muốn)
- **Batch Size**: 16 (P100) hoặc 12 (T4)
- **Learning Rate**: 2e-5
- **Time**: ~20-40 phút

In [ ]:
# Training với GPU
# P100: batch_size=16, T4: batch_size=12

!python train_phobert.py --epochs 3 --batch_size 16

# Nếu bị OOM (Out of Memory), giảm batch_size:
# !python train_phobert.py --epochs 3 --batch_size 12
# hoặc
# !python train_phobert.py --epochs 3 --batch_size 8

## 📊 Bước 6: Kiểm Tra Kết Quả

In [ ]:
# Xem training log
import os

log_file = "models/phobert_finetuned/training.log"

if os.path.exists(log_file):
    print("📄 Training Log:")
    print("="*60)
    with open(log_file, 'r', encoding='utf-8') as f:
        print(f.read())
    print("="*60)
else:
    print("⚠️  Log file not found")

# Xem các file đã tạo
print("\n📁 Model files:")
!ls -lh models/phobert_finetuned/

## 📦 Bước 7: Tạo ZIP File để Download

In [ ]:
# Tạo zip file
!zip -r phobert_finetuned.zip models/phobert_finetuned/

# Kiểm tra kích thước
import os
zip_size = os.path.getsize('phobert_finetuned.zip') / (1024 * 1024)
print(f"\n✅ Zip file created: phobert_finetuned.zip ({zip_size:.1f} MB)")
print("\n📥 Download file từ Output panel bên phải →")

## 🧪 Bước 8: Test Model (Optional)

In [ ]:
# Quick test model
from core_nlp.phobert_model import PhoBERTNLPPipeline

# Load fine-tuned model
pipeline = PhoBERTNLPPipeline(model_path="./models/phobert_finetuned")

# Test cases
test_cases = [
    "mai 8h họp ở phòng 302",
    "thứ 5 tuần sau đi chơi",
    "15/11 sinh nhật nhắc trước 2 ngày",
    "t7 này 14h30 xem phim tại CGV"
]

print("\n🧪 Testing fine-tuned model:")
print("="*60)

for text in test_cases:
    result = pipeline.extract(text)
    print(f"\n📝 Input: {text}")
    print(f"Event: {result.get('event', 'N/A')}")
    print(f"Time: {result.get('start_time', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Reminder: {result.get('reminder_minutes', 0)} minutes")
    print("-"*60)

## 📥 Sau Khi Download Model:

### 1. Giải nén file
```bash
# Windows PowerShell
Expand-Archive -Path phobert_finetuned.zip -DestinationPath .

# Linux/Mac
unzip phobert_finetuned.zip
```

### 2. Test model local
```bash
python comprehensive_test.py
```

### 3. Commit lên GitHub
```bash
git add models/phobert_finetuned
git commit -m "v1.1.0: Add fine-tuned PhoBERT model

- Trained on Kaggle P100 GPU
- 76K+ training samples
- 3 epochs, ~25 minutes
- Expected Macro F1: 90%+"

git push origin main
```

### 4. Update README
```bash
# Add to README.md:
- ✅ PhoBERT fine-tuned model (90%+ F1)
- ✅ Hybrid pipeline with PhoBERT support
```